In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda

In [25]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images.shape, test_labels.shape

((60000, 28, 28), (10000,))

In [26]:
train_images = train_images.reshape((-1, 28, 28, 1)).astype('float32')
test_images = test_images.reshape((-1, 28, 28, 1)).astype('float32')

In [27]:
train_images /= 255.0
test_images /= 255.0

In [28]:
class SafeModel(tf.keras.Model):
    def __init__(self):
        super(SafeModel, self).__init__()
        self.conv1 = Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))
        self.pool1 = MaxPooling2D((2, 2))
        self.conv2 = Conv2D(64, (3, 3), activation='relu')
        self.pool2 = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.dropout = Dropout(0.5)
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout(x)
        return self.d2(x)

In [29]:
safe = SafeModel()
safe.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
safe.fit(train_images, train_labels, epochs=1)
safe.save('models/safemodel')

1875/1875 [==============================] - 5s 3ms/step - loss: 0.5481 - accuracy: 0.8056
INFO:tensorflow:Assets written to: models/safemodel\assets


INFO:tensorflow:Assets written to: models/safemodel\assets


In [30]:
from modelscan.modelscan import ModelScan

ModelScan().scan('models/safemodel')

{'summary': {'total_issues_by_severity': {'LOW': 0,
   'MEDIUM': 0,
   'HIGH': 0,
   'CRITICAL': 0},
  'total_issues': 0,
  'input_path': 'models/safemodel',
  'absolute_path': 'C:\\Users\\nikos\\PycharmProjects\\ITHACA\\models\\safemodel',
  'modelscan_version': '0.7.6',
  'timestamp': '2024-07-12T16:40:54.979203',
  'scanned': {'total_scanned': 3,
   'scanned_files': ['fingerprint.pb', 'keras_metadata.pb', 'saved_model.pb']},
  'skipped': {'total_skipped': 2,
   'skipped_files': [{'category': 'SCAN_NOT_SUPPORTED',
     'description': 'Model Scan did not scan file',
     'source': 'variables\\variables.data-00000-of-00001'},
    {'category': 'SCAN_NOT_SUPPORTED',
     'description': 'Model Scan did not scan file',
     'source': 'variables\\variables.index'}]}},
 'issues': [],
 'errors': []}

In [41]:
import os

In [57]:
class IOModel(tf.keras.Model):
    def __init__(self):
        super(IOModel, self).__init__()
        self.conv1 = Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))
        self.pool1 = MaxPooling2D((2, 2))
        self.conv2 = Conv2D(64, (3, 3), activation='relu')
        self.pool2 = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.dropout = Dropout(0.5)
        self.d2 = Dense(10, activation='softmax')

    def call(self, x, training=None, mask=None):
        f = tf.io.read_file('top_secret.txt')
        # add a lambda layer to read the file
        g = tf.io.write_file('top_secret.txt', tf.constant('top secret'))
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout(x)
        return self.d2(x)

In [58]:
io = IOModel()
io.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
io.fit(train_images, train_labels, epochs=1)
io.save('models/iomodel')

1875/1875 [==============================] - 7s 4ms/step - loss: 0.5702 - accuracy: 0.7925
INFO:tensorflow:Assets written to: models/iomodel\assets


INFO:tensorflow:Assets written to: models/iomodel\assets


In [62]:
results = ModelScan().scan('models/iomodel')
results

{'summary': {'total_issues_by_severity': {'LOW': 0,
   'MEDIUM': 0,
   'HIGH': 2,
   'CRITICAL': 0},
  'total_issues': 2,
  'input_path': 'models/iomodel',
  'absolute_path': 'C:\\Users\\nikos\\PycharmProjects\\ITHACA\\models\\iomodel',
  'modelscan_version': '0.7.6',
  'timestamp': '2024-07-12T16:50:37.228844',
  'scanned': {'total_scanned': 3,
   'scanned_files': ['fingerprint.pb', 'keras_metadata.pb', 'saved_model.pb']},
  'skipped': {'total_skipped': 2,
   'skipped_files': [{'category': 'SCAN_NOT_SUPPORTED',
     'description': 'Model Scan did not scan file',
     'source': 'variables\\variables.data-00000-of-00001'},
    {'category': 'SCAN_NOT_SUPPORTED',
     'description': 'Model Scan did not scan file',
     'source': 'variables\\variables.index'}]}},
 'issues': [{'description': "Use of unsafe operator 'ReadFile' from module 'Tensorflow'",
   'operator': 'ReadFile',
   'module': 'Tensorflow',
   'source': 'saved_model.pb',
   'scanner': 'modelscan.scanners.SavedModelTensorflowO

In [65]:
results['summary']['total_issues_by_severity']

{'LOW': 0, 'MEDIUM': 0, 'HIGH': 2, 'CRITICAL': 0}

In [69]:
# create a dataframes from the results
import pandas as pd
df = pd.DataFrame(results['issues'], columns=['severity', 'operator', 'description'])
df.head(10)

,severity,operator,description
0,HIGH,ReadFile,Use of unsafe operator 'ReadFile' from module ...
1,HIGH,WriteFile,Use of unsafe operator 'WriteFile' from module...
